In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from skimage import filters
from skimage import feature
from skimage import morphology
from skimage import color
from scipy import ndimage
import matplotlib.pyplot as plt
import cv2
import numpy as np
from google.colab import files
from skimage.metrics import structural_similarity as ssim
from math import log10, sqrt

m=3
n=3
box_filter = np.ones((m, n), dtype=np.float32) / (m*n)

# 1. Inserto una imagen RGB y el valor de umbralización y devuelve esa imagen umbralizada en escala de grises.
def umbralizacion_imagen(imagen, umbral):
    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    _, umbralizada = cv2.threshold(gris, umbral, 255, cv2.THRESH_BINARY)
    return umbralizada

# 2. Inserto una imagen RGB y mediante el metodo de OTSU me devuelve el valor de umbralización mejor.
def umbralizacion_otsu(imagen):
    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    # Aplica el método de Otsu y obtiene el umbral
    _, umbralizado_otsu = cv2.threshold(gris, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Devuelve la imagen umbralizada y el valor del umbral
    return umbralizado_otsu, _

# 3. Inserto una imagen en RGB y plotea los histogramas en escala RGB y escala de grises.
def histogramas(imagen):
    # Histogramas en RGB
    colores = ('b', 'g', 'r')
    plt.figure(figsize=(10, 5))

    for i, color in enumerate(colores):
        hist = cv2.calcHist([imagen], [i], None, [256], [0, 256])
        plt.bar(range(256), hist.flatten(), color=color, width=1)

    plt.title('Histogramas RGB')
    plt.xlabel('Intensidad')
    plt.ylabel('Frecuencia')
    plt.show()

    # Histograma en escala de grises
    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    hist_gris = cv2.calcHist([gris], [0], None, [256], [0, 256])

    plt.bar(range(256), hist_gris.flatten(), color='k', width=1)
    plt.title('Histograma en escala de grises')
    plt.xlabel('Intensidad')
    plt.ylabel('Frecuencia')
    plt.show()


# 4. Inserto una imagen y devuelve una imagen con la funcion erode.
def erode(imagen, kernel_size=(3, 3)):
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.erode(imagen, kernel, iterations=1)

# 5. Inserto una imagen y devuelve una imagen con la funcion dilate.
def dilate(imagen, kernel_size=(3, 3)):
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.dilate(imagen, kernel, iterations=1)

# 6. Inserto una imagen y devuelve una imagen con la funcion open.
def open(imagen, kernel_size=(3, 3)):
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.morphologyEx(imagen, cv2.MORPH_OPEN, kernel)

# 7. Inserto una imagen y devuelve una imagen con la funcion close.
def close(imagen, kernel_size=(3, 3)):
    kernel = np.ones(kernel_size, np.uint8)
    return cv2.morphologyEx(imagen, cv2.MORPH_CLOSE, kernel)

# 8. Inserto una imagen y devuelve una imagen tras un filtro paso bajo.
def filtro_paso_bajo(imagen, tamaño_kernel=5):
    return cv2.GaussianBlur(imagen, (tamaño_kernel, tamaño_kernel), 0)

# 9. Inserto una imagen y devuelve una imagen tras un filtro paso alto.
def filtro_paso_alto(imagen, tamaño_kernel=5):
    imagen_baja = cv2.GaussianBlur(imagen, (tamaño_kernel, tamaño_kernel), 0)
    return cv2.subtract(imagen, imagen_baja)

# 10. Inserto una imagen y devuelve una imagen con detección de bordes con el operador Roberts.
def bordes_roberts(imagen):
    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    roberts = filters.roberts(gris)
    return (roberts * 255).astype(np.uint8)

# 11. Inserto una imagen y devuelve una imagen con detección de bordes con el operador Prewitt.
def bordes_prewitt(imagen):
    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    prewitt = filters.prewitt(gris)
    return (prewitt * 255).astype(np.uint8)

# 12. Inserto una imagen y devuelve una imagen con detección de bordes con el operador Sobel.
def bordes_sobel(imagen):
    #gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    sobel = filters.sobel(imagen)
    return (sobel * 255).astype(np.uint8)

# 13. Inserto una imagen y devuelve una imagen con detección de bordes con el operador Canny.
def bordes_canny(imagen, umbral1=100, umbral2=200):
    if len(imagen.shape) == 3:
      gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    else:
      gris = imagen
    canny = cv2.Canny(gris, umbral1, umbral2)
    return canny

# 14. Recorta una imagen y devuelve la zona de interés a tratar.
def crop_image(img):
  if len(img.shape) == 3:
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  else:
    gray_img = img
  threshold = ((gray_img > np.mean(img))*255).astype(np.uint8)
  contours, hierarchy = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
  if not contours:
    return img
  c = max(contours, key = cv2.contourArea)
  x,y,w,h = cv2.boundingRect(c)
  cropped_img = img[y:y+h, x:x+w]
  return cropped_img

# 15. Superpone los bordes detectados sobre la imagen.
def superponer_bordes(imagen, bordes, color_borde=[255, 0, 0]):
  if len(imagen.shape) == 2:
    overlay = cv2.cvtColor(imagen, cv2.COLOR_GRAY2BGR)
  else:
    overlay = imagen.copy()
  overlay[bordes > 0] = color_borde
  return overlay

# 16. Inserto una imagen y devuelve una imagen tras un filtro box filter.
def filtro_box(imagen):
  return cv2.filter2D(imagen, ddepth=-1, kernel=box_filter)

# 17. Inserto una imagen y devuele una imagen tras un gradiente de Beucher
def gradient_beucher(imagen, kernel_size=25):
  if len(imagen.shape) == 3:
    imagen = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_size, kernel_size))
  closed_image = cv2.morphologyEx(imagen, cv2.MORPH_CLOSE, kernel)

# 18. Calcula la métrica de índice de calidad estructural (SSIM)
def calcular_ssim(imagen_original, imagen_filtrada):
  imagen_original = np.uint8(imagen_original)
  imagen_filtrada = np.uint8(imagen_filtrada)
  return ssim(imagen_original, imagen_filtrada, data_range=255)

# 19. Calcula la métrica de pico de relación señal-ruido (PSNR)
def calcular_psnr(imagen_original, imagen_filtrada):
  imagen_original = np.uint8(imagen_original)
  imagen_filtrada = np.uint8(imagen_filtrada)
  return cv2.PSNR(imagen_original, imagen_filtrada)

### IMAGENES CANCER


In [ ]:
imagen_mamografia_cancer = cv2.imread("/content/drive/MyDrive/VISION ARTIFICIAL ACTIVIDAD 2/imágenes/mamografia_cancer.jpg")
imagen_mamografia_no_cancer = cv2.imread("/content/drive/MyDrive/VISION ARTIFICIAL ACTIVIDAD 2/imágenes/mamografia_no_cancer.jpg")
plt.figure(figsize=(14, 5))

# Mostrar resultados
plt.subplot(1, 2, 1)
plt.imshow(imagen_mamografia_cancer)
plt.title("Mamografia con cancer")
plt.axis("off")

plt.subplot(1, 2, 2)

plt.imshow(imagen_mamografia_no_cancer)
plt.title("Mamografia sin cancer")
plt.axis("off")
plt.tight_layout()
plt.show()

Se realiza filtro de Sobel tanto en X como en Y

In [ ]:
# Definir los filtros paso-alto (kernels) de la diapositiva
kernels = {
    "Filtro Sobel X": np.array([[1,  0,  -1],
                                [2,  0,  -2],
                                [1,  0,  -1]]),

    "Filtro Sobel Y": np.array([[1,  2,  1],
                                [0,  0,  0],
                                [-1, -2, -1]]),
}

# Aplicar los filtros a ambas imágenes y guardar los resultados
filtered_images_cancer = {}
filtered_images_no_cancer = {}

if len(imagen_mamografia_cancer.shape) > 2:
  imagen_mamografia_cancer = cv2.cvtColor(imagen_mamografia_cancer, cv2.COLOR_BGR2GRAY)
if len(imagen_mamografia_no_cancer.shape) > 2:
  imagen_mamografia_no_cancer = cv2.cvtColor(imagen_mamografia_no_cancer, cv2.COLOR_BGR2GRAY)

for name, kernel in kernels.items():
    # Filtrar imagen de mamografía con cáncer
    filtered_images_cancer[name] = cv2.filter2D(imagen_mamografia_cancer, ddepth=-1, kernel=kernel)
    # Filtrar imagen de mamografía sin cáncer
    filtered_images_no_cancer[name] = cv2.filter2D(imagen_mamografia_no_cancer, ddepth=-1, kernel=kernel)

# Mostrar las imágenes originales y filtradas
plt.figure(figsize=(8, 6))

# Imagen original con cáncer
plt.subplot(2, 3, 1)
plt.title("Original - Sin Cáncer", fontsize=12)
plt.imshow(imagen_mamografia_cancer, cmap='gray')
plt.axis('off')

# Imagen original sin cáncer
plt.subplot(2, 3, 4)
plt.title("Original - Cáncer", fontsize=12)
plt.imshow(imagen_mamografia_no_cancer, cmap='gray')
plt.axis('off')

# Resultados de los filtros para cada imagen
for i, (name, filtered_image) in enumerate(filtered_images_cancer.items(), start=2):
    # Filtros aplicados a la imagen con cáncer
    plt.subplot(2, 3, i)
    plt.title(f"{name} - cáncer", fontsize=12)
    plt.imshow(filtered_image, cmap='gray')
    plt.axis('off')

for i, (name, filtered_image) in enumerate(filtered_images_no_cancer.items(), start=5):
    # Filtros aplicados a la imagen sin cáncer
    plt.subplot(2, 3, i)
    plt.title(f"{name} - Sin Cáncer", fontsize=12)
    plt.imshow(filtered_image, cmap='gray')
    plt.axis('off')

plt.tight_layout()
plt.show()


ssim_values_cancer = {}
psnr_values_cancer = {}
ssim_values_no_cancer = {}
psnr_values_no_cancer = {}

for name, filtered_image in filtered_images_cancer.items():
  ssim_values_cancer[name] = calcular_ssim(imagen_mamografia_cancer, filtered_image)
  psnr_values_cancer[name] = calcular_psnr(imagen_mamografia_cancer, filtered_image)


for name, filtered_image in filtered_images_no_cancer.items():
  ssim_values_no_cancer[name] = calcular_ssim(imagen_mamografia_no_cancer, filtered_image)
  psnr_values_no_cancer[name] = calcular_psnr(imagen_mamografia_no_cancer, filtered_image)


# Mostrar resultados SSIM
for name, ssim_value in ssim_values_cancer.items():
  psnr_value = psnr_values_cancer[name]
  print(f"{name} - Cáncer: SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

for name, ssim_value in ssim_values_no_cancer.items():
  psnr_value = psnr_values_no_cancer[name]
  print(f"{name} - Sin Cáncer: SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

Filtros diagonales

In [ ]:
# Definir los filtros paso-alto (kernels) de la diapositiva
c = 4
kernels = {
    "Filtro Diagonal 135º": np.array([[-1, -1,   c],
                                      [-1,  c,  -1],
                                      [ c, -1,  -1]]),

    "Filtro Diagonal 45º": np.array([[  c, -1,  -1],
                                     [ -1,  c,  -1],
                                     [ -1, -1,  c]]),
}

# Aplicar los filtros a ambas imágenes y guardar los resultados
filtered_images_cancer = {}
filtered_images_no_cancer = {}

for name, kernel in kernels.items():
    # Filtrar imagen de mamografía con cáncer
    filtered_images_cancer[name] = cv2.filter2D(imagen_mamografia_cancer, ddepth=-1, kernel=kernel)
    # Filtrar imagen de mamografía sin cáncer
    filtered_images_no_cancer[name] = cv2.filter2D(imagen_mamografia_no_cancer, ddepth=-1, kernel=kernel)

# Mostrar las imágenes originales y filtradas
plt.figure(figsize=(8, 6))

# Imagen original con cáncer
plt.subplot(2, 3, 1)
plt.title("Original - Cáncer", fontsize=12)
plt.imshow(imagen_mamografia_cancer, cmap='gray')
plt.axis('off')

# Imagen original sin cáncer
plt.subplot(2, 3, 4)
plt.title("Original - Sin Cáncer", fontsize=12)
plt.imshow(imagen_mamografia_no_cancer, cmap='gray')
plt.axis('off')

# Resultados de los filtros para cada imagen
for i, (name, filtered_image) in enumerate(filtered_images_cancer.items(), start=2):
    # Filtros aplicados a la imagen con cáncer
    plt.subplot(2, 3, i)
    plt.title(f"{name} - Cáncer", fontsize=12)
    plt.imshow(filtered_image, cmap='gray')
    plt.axis('off')

for i, (name, filtered_image) in enumerate(filtered_images_no_cancer.items(), start=5):
    # Filtros aplicados a la imagen sin cáncer
    plt.subplot(2, 3, i)
    plt.title(f"{name} - Sin Cáncer", fontsize=12)
    plt.imshow(filtered_image, cmap='gray')
    plt.axis('off')

plt.tight_layout()
plt.show()

ssim_values_cancer = {}
psnr_values_cancer = {}
ssim_values_no_cancer = {}
psnr_values_no_cancer = {}

for name, filtered_image in filtered_images_cancer.items():
  ssim_values_cancer[name] = calcular_ssim(imagen_mamografia_cancer, filtered_image)
  psnr_values_cancer[name] = calcular_psnr(imagen_mamografia_cancer, filtered_image)


for name, filtered_image in filtered_images_no_cancer.items():
  ssim_values_no_cancer[name] = calcular_ssim(imagen_mamografia_no_cancer, filtered_image)
  psnr_values_no_cancer[name] = calcular_psnr(imagen_mamografia_no_cancer, filtered_image)


# Mostrar resultados SSIM
for name, ssim_value in ssim_values_cancer.items():
  psnr_value = psnr_values_cancer[name]
  print(f"{name} - Cáncer: SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

for name, ssim_value in ssim_values_no_cancer.items():
  psnr_value = psnr_values_no_cancer[name]
  print(f"{name} - Sin Cáncer: SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

Filtro laplaciano

In [ ]:
# Definir los filtros paso-alto (kernels) de la diapositiva
c = 8
kernels = {
    "Filtro Laplaciano": np.array([[ 1,  1,   1],
                                   [ 1, -c,   1],
                                   [ 1,  1,   1]]),
}

# Aplicar los filtros a ambas imágenes y guardar los resultados
filtered_images_cancer = {}
filtered_images_no_cancer = {}

for name, kernel in kernels.items():
    # Filtrar imagen de mamografía con cáncer
    filtered_images_cancer[name] = cv2.filter2D(imagen_mamografia_cancer, ddepth=-1, kernel=kernel)
    # Filtrar imagen de mamografía sin cáncer
    filtered_images_no_cancer[name] = cv2.filter2D(imagen_mamografia_no_cancer, ddepth=-1, kernel=kernel)

# Mostrar las imágenes originales y filtradas
plt.figure(figsize=(12, 8))

# Imagen original con cáncer
plt.subplot(2, 2, 1)
plt.title("Original - Cáncer", fontsize=12)
plt.imshow(imagen_mamografia_cancer, cmap='gray')
plt.axis('off')

# Imagen original sin cáncer
plt.subplot(2, 2, 3)
plt.title("Original - Sin Cáncer", fontsize=12)
plt.imshow(imagen_mamografia_no_cancer, cmap='gray')
plt.axis('off')

# Filtro aplicado a la imagen con cáncer
for i, (name, filtered_image) in enumerate(filtered_images_cancer.items(), start=2):
    plt.subplot(2, 2, 2)
    plt.title(f"{name} - Cáncer", fontsize=12)
    plt.imshow(filtered_image, cmap='gray')
    plt.axis('off')

# Filtro aplicado a la imagen sin cáncer
for i, (name, filtered_image) in enumerate(filtered_images_no_cancer.items(), start=4):
    plt.subplot(2, 2, 4)
    plt.title(f"{name} - Sin Cáncer", fontsize=12)
    plt.imshow(filtered_image, cmap='gray')
    plt.axis('off')

plt.tight_layout()
plt.show()

ssim_values_cancer = {}
psnr_values_cancer = {}
ssim_values_no_cancer = {}
psnr_values_no_cancer = {}

for name, filtered_image in filtered_images_cancer.items():
  ssim_values_cancer[name] = calcular_ssim(imagen_mamografia_cancer, filtered_image)
  psnr_values_cancer[name] = calcular_psnr(imagen_mamografia_cancer, filtered_image)


for name, filtered_image in filtered_images_no_cancer.items():
  ssim_values_no_cancer[name] = calcular_ssim(imagen_mamografia_no_cancer, filtered_image)
  psnr_values_no_cancer[name] = calcular_psnr(imagen_mamografia_no_cancer, filtered_image)


# Mostrar resultados SSIM
for name, ssim_value in ssim_values_cancer.items():
  psnr_value = psnr_values_cancer[name]
  print(f"{name} - Cáncer: SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

for name, ssim_value in ssim_values_no_cancer.items():
  psnr_value = psnr_values_no_cancer[name]
  print(f"{name} - Sin Cáncer: SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

Operador de Prewit

In [ ]:
# Normalizar ambas imágenes a rango 0-255 si es necesario
image_cancer = cv2.normalize(imagen_mamografia_cancer, None, 0, 255, cv2.NORM_MINMAX)
image_no_cancer = cv2.normalize(imagen_mamografia_no_cancer, None, 0, 255, cv2.NORM_MINMAX)

if len(image_cancer.shape) > 2:
  image_cancer = cv2.cvtColor(image_cancer, cv2.COLOR_BGR2GRAY)
if len(image_no_cancer.shape) > 2:
  image_no_cancer = cv2.cvtColor(image_no_cancer, cv2.COLOR_BGR2GRAY)

# Definir los kernels del operador de Prewitt
kernel_x = np.array([[-1, 0, 1],
                     [-1, 0, 1],
                     [-1, 0, 1]], dtype=np.float32)

kernel_y = np.array([[ 1,  1,  1],
                     [ 0,  0,  0],
                     [-1, -1, -1]], dtype=np.float32)

# Función para calcular el gradiente de Prewitt y su magnitud
def apply_prewitt(image):
    prewitt_x = cv2.filter2D(image, cv2.CV_64F, kernel_x)
    prewitt_y = cv2.filter2D(image, cv2.CV_64F, kernel_y)
    magnitude = np.sqrt(prewitt_x ** 2 + prewitt_y ** 2)
    magnitude = np.uint8(magnitude / magnitude.max() * 255)  # Escalar a rango 0-255
    return prewitt_x, prewitt_y, magnitude



# Aplicar Prewitt a ambas imágenes
prewitt_x_cancer, prewitt_y_cancer, magnitude_cancer = apply_prewitt(image_cancer)
prewitt_x_no_cancer, prewitt_y_no_cancer, magnitude_no_cancer = apply_prewitt(image_no_cancer)

# Mostrar los resultados
plt.figure(figsize=(16, 12))

# Imagen con cáncer
plt.subplot(2, 4, 1)
plt.title("Original - Cáncer")
plt.imshow(image_cancer, cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 2)
plt.title("Prewitt X - Cáncer")
plt.imshow(np.abs(prewitt_x_cancer), cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 3)
plt.title("Prewitt Y - Cáncer")
plt.imshow(np.abs(prewitt_y_cancer), cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 4)
plt.title("Magnitud - Cáncer")
plt.imshow(magnitude_cancer, cmap="gray")
plt.axis("off")

# Imagen sin cáncer
plt.subplot(2, 4, 5)
plt.title("Original - Sin Cáncer")
plt.imshow(image_no_cancer, cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 6)
plt.title("Prewitt X - Sin Cáncer")
plt.imshow(np.abs(prewitt_x_no_cancer), cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 7)
plt.title("Prewitt Y - Sin Cáncer")
plt.imshow(np.abs(prewitt_y_no_cancer), cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 8)
plt.title("Magnitud - Sin Cáncer")
plt.imshow(magnitude_no_cancer, cmap="gray")
plt.axis("off")

plt.tight_layout()
plt.show()

# Calcular SSIM y PSNR para las imágenes de Prewitt
ssim_values_prewitt_cancer = {}
psnr_values_prewitt_cancer = {}

ssim_values_prewitt_no_cancer = {}
psnr_values_prewitt_no_cancer = {}

# Para imágenes con cáncer
ssim_values_prewitt_cancer['Prewitt X'] = calcular_ssim(image_cancer, np.abs(prewitt_x_cancer))
psnr_values_prewitt_cancer['Prewitt X'] = calcular_psnr(image_cancer, np.abs(prewitt_x_cancer))

ssim_values_prewitt_cancer['Prewitt Y'] = calcular_ssim(image_cancer, np.abs(prewitt_y_cancer))
psnr_values_prewitt_cancer['Prewitt Y'] = calcular_psnr(image_cancer, np.abs(prewitt_y_cancer))

ssim_values_prewitt_cancer['Magnitud'] = calcular_ssim(image_cancer, magnitude_cancer)
psnr_values_prewitt_cancer['Magnitud'] = calcular_psnr(image_cancer, magnitude_cancer)

# Para imágenes sin cáncer
ssim_values_prewitt_no_cancer['Prewitt X'] = calcular_ssim(image_no_cancer, np.abs(prewitt_x_no_cancer))
psnr_values_prewitt_no_cancer['Prewitt X'] = calcular_psnr(image_no_cancer, np.abs(prewitt_x_no_cancer))

ssim_values_prewitt_no_cancer['Prewitt Y'] = calcular_ssim(image_no_cancer, np.abs(prewitt_y_no_cancer))
psnr_values_prewitt_no_cancer['Prewitt Y'] = calcular_psnr(image_no_cancer, np.abs(prewitt_y_no_cancer))

ssim_values_prewitt_no_cancer['Magnitud'] = calcular_ssim(image_no_cancer, magnitude_no_cancer)
psnr_values_prewitt_no_cancer['Magnitud'] = calcular_psnr(image_no_cancer, magnitude_no_cancer)

# Mostrar los resultados SSIM y PSNR
print("SSIM y PSNR para imagen con cáncer:")
for name in ssim_values_prewitt_cancer:
    print(f"{name} - Cáncer: SSIM = {ssim_values_prewitt_cancer[name]:.4f}, PSNR = {psnr_values_prewitt_cancer[name]:.4f} dB")

print("\nSSIM y PSNR para imagen sin cáncer:")
for name in ssim_values_prewitt_no_cancer:
    print(f"{name} - Sin Cáncer: SSIM = {ssim_values_prewitt_no_cancer[name]:.4f}, PSNR = {psnr_values_prewitt_no_cancer[name]:.4f} dB")

In [ ]:
# Normalizar ambas imágenes a rango 0-255 si es necesario
image_cancer = cv2.normalize(imagen_mamografia_cancer, None, 0, 255, cv2.NORM_MINMAX)
image_no_cancer = cv2.normalize(imagen_mamografia_no_cancer, None, 0, 255, cv2.NORM_MINMAX)

# Definir los kernels del operador de Prewitt
kernel_x = np.array([[-1, 0, 1],
                     [-1, 0, 1],
                     [-1, 0, 1]], dtype=np.float32)

kernel_y = np.array([[ 1,  1,  1],
                     [ 0,  0,  0],
                     [-1, -1, -1]], dtype=np.float32)

# Función para calcular el gradiente de Prewitt y su magnitud
def apply_prewitt(image):
    prewitt_x = cv2.filter2D(image, cv2.CV_64F, kernel_x)
    prewitt_y = cv2.filter2D(image, cv2.CV_64F, kernel_y)
    magnitude = np.sqrt(prewitt_x ** 2 + prewitt_y ** 2)
    magnitude = np.uint8(magnitude / magnitude.max() * 255)  # Escalar a rango 0-255
    return prewitt_x, prewitt_y, magnitude

# Aplicar Prewitt a ambas imágenes
prewitt_x_cancer, prewitt_y_cancer, magnitude_cancer = apply_prewitt(image_cancer)
prewitt_x_no_cancer, prewitt_y_no_cancer, magnitude_no_cancer = apply_prewitt(image_no_cancer)

# Mostrar los resultados
plt.figure(figsize=(16, 12))

# Imagen con cáncer
plt.subplot(2, 4, 1)
plt.title("Original - Cáncer")
plt.imshow(image_cancer, cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 2)
plt.title("Prewitt X - Cáncer")
plt.imshow(np.abs(prewitt_x_cancer), cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 3)
plt.title("Prewitt Y - Cáncer")
plt.imshow(np.abs(prewitt_y_cancer), cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 4)
plt.title("Magnitud - Cáncer")
plt.imshow(magnitude_cancer, cmap="gray")
plt.axis("off")

# Imagen sin cáncer
plt.subplot(2, 4, 5)
plt.title("Original - Sin Cáncer")
plt.imshow(image_no_cancer, cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 6)
plt.title("Prewitt X - Sin Cáncer")
plt.imshow(np.abs(prewitt_x_no_cancer), cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 7)
plt.title("Prewitt Y - Sin Cáncer")
plt.imshow(np.abs(prewitt_y_no_cancer), cmap="gray")
plt.axis("off")

plt.subplot(2, 4, 8)
plt.title("Magnitud - Sin Cáncer")
plt.imshow(magnitude_no_cancer, cmap="gray")
plt.axis("off")

plt.tight_layout()
plt.show()


Erosión con distintos kernel

In [ ]:
# Crear un kernel circular para la erosión
kernel_size = 35
kernel_circular = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))

# Crear un kernel rectangular en la dirección X (horizontal)
kernel_horizontal = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 10))

# Crear un kernel rectangular en la dirección Y (vertical)
kernel_vertical = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 25))

# Función para aplicar diferentes kernels de erosión
def apply_erosion(image):
    eroded_circular = cv2.erode(image, kernel_circular, iterations=1)
    eroded_horizontal = cv2.erode(image, kernel_horizontal, iterations=1)
    eroded_vertical = cv2.erode(image, kernel_vertical, iterations=1)
    return eroded_circular, eroded_horizontal, eroded_vertical

# Aplicar erosión a ambas imágenes
eroded_circular_cancer, eroded_horizontal_cancer, eroded_vertical_cancer = apply_erosion(imagen_mamografia_cancer)
eroded_circular_no_cancer, eroded_horizontal_no_cancer, eroded_vertical_no_cancer = apply_erosion(imagen_mamografia_no_cancer)

# Mostrar todas las imágenes
plt.figure(figsize=(12, 12))

# Resultados para imagen con cáncer
plt.subplot(3, 3, 1)
plt.title("Original - Cáncer")
plt.imshow(imagen_mamografia_cancer, cmap='gray')
plt.axis('off')

plt.subplot(3, 3, 2)
plt.title("Erosión Circular - Cáncer")
plt.imshow(eroded_circular_cancer, cmap='gray')
plt.axis('off')

plt.subplot(3, 3, 3)
plt.title("Erosión Horizontal - Cáncer")
plt.imshow(eroded_horizontal_cancer, cmap='gray')
plt.axis('off')

plt.subplot(3, 3, 4)
plt.title("Erosión Vertical - Cáncer")
plt.imshow(eroded_vertical_cancer, cmap='gray')
plt.axis('off')

plt.subplot(3, 3, 5)
plt.title("Original - Sin Cáncer")
plt.imshow(imagen_mamografia_no_cancer, cmap='gray')
plt.axis('off')

plt.subplot(3, 3, 6)
plt.title("Erosión Circular - Sin Cáncer")
plt.imshow(eroded_circular_no_cancer, cmap='gray')
plt.axis('off')

plt.subplot(3, 3, 7)
plt.title("Erosión Horizontal - Sin Cáncer")
plt.imshow(eroded_horizontal_no_cancer, cmap='gray')
plt.axis('off')

plt.subplot(3, 3, 8)
plt.title("Erosión Vertical -  Sin Cáncer")
plt.imshow(eroded_vertical_no_cancer, cmap='gray')
plt.axis('off')

plt.tight_layout()
plt.show()


Erosión + dilatación = Apertura

In [ ]:
# Definir un kernel (por ejemplo, rectangular de tamaño 5x5)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# Función para aplicar erosión seguida de dilatación
def erode_and_dilate(image, kernel):
    eroded = cv2.erode(image, kernel, iterations=1)
    dilated = cv2.dilate(eroded, kernel, iterations=1)
    return eroded, dilated

# Aplicar el proceso a ambas imágenes
eroded_cancer, dilated_cancer = erode_and_dilate(imagen_mamografia_cancer, kernel)
eroded_no_cancer, dilated_no_cancer = erode_and_dilate(imagen_mamografia_no_cancer, kernel)

# Visualización de los resultados
fig, axs = plt.subplots(2, 3, figsize=(12, 8))

# Resultados para la imagen con cáncer
axs[0, 0].imshow(imagen_mamografia_cancer, cmap='gray', interpolation='nearest')
axs[0, 0].set_title("Original - Cáncer")
axs[0, 0].axis('off')

axs[0, 1].imshow(eroded_cancer, cmap='gray', interpolation='nearest')
axs[0, 1].set_title("Erosión - Cáncer")
axs[0, 1].axis('off')

axs[0, 2].imshow(dilated_cancer, cmap='gray', interpolation='nearest')
axs[0, 2].set_title("Dilatar Erosión - Cáncer")
axs[0, 2].axis('off')

# Resultados para la imagen sin cáncer
axs[1, 0].imshow(imagen_mamografia_no_cancer, cmap='gray', interpolation='nearest')
axs[1, 0].set_title("Original - Sin Cáncer")
axs[1, 0].axis('off')

axs[1, 1].imshow(eroded_no_cancer, cmap='gray', interpolation='nearest')
axs[1, 1].set_title("Erosión - Sin Cáncer")
axs[1, 1].axis('off')

axs[1, 2].imshow(dilated_no_cancer, cmap='gray', interpolation='nearest')
axs[1, 2].set_title("Dilatar Erosión - Sin Cáncer")
axs[1, 2].axis('off')

plt.tight_layout()
plt.show()


Erosión + Dilatación = Cerradura

In [ ]:
# Definir un kernel (por ejemplo, rectangular de tamaño 5x5)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))

# Función para aplicar dilatación seguida de erosión
def dilate_and_erode(image, kernel):
    dilated = cv2.dilate(image, kernel, iterations=1)
    eroded = cv2.erode(dilated, kernel, iterations=1)
    return dilated, eroded

# Aplicar el proceso a ambas imágenes
dilated_cancer, eroded_cancer = dilate_and_erode(imagen_mamografia_cancer, kernel)
dilated_no_cancer, eroded_no_cancer = dilate_and_erode(imagen_mamografia_no_cancer, kernel)

# Visualización de los resultados
fig, axs = plt.subplots(2, 3, figsize=(12, 10))

# Resultados para la imagen con cáncer
axs[0, 0].imshow(imagen_mamografia_cancer, cmap='gray', interpolation='nearest')
axs[0, 0].set_title("Original - Cáncer")
axs[0, 0].axis('off')

axs[0, 1].imshow(dilated_cancer, cmap='gray', interpolation='nearest')
axs[0, 1].set_title("1º Dilatar - Cáncer")
axs[0, 1].axis('off')

axs[0, 2].imshow(eroded_cancer, cmap='gray', interpolation='nearest')
axs[0, 2].set_title("2º Erosión - Cáncer")
axs[0, 2].axis('off')

# Resultados para la imagen sin cáncer
axs[1, 0].imshow(imagen_mamografia_no_cancer, cmap='gray', interpolation='nearest')
axs[1, 0].set_title("Original - Sin Cáncer")
axs[1, 0].axis('off')

axs[1, 1].imshow(dilated_no_cancer, cmap='gray', interpolation='nearest')
axs[1, 1].set_title("1º Dilatar - Sin Cáncer")
axs[1, 1].axis('off')

axs[1, 2].imshow(eroded_no_cancer, cmap='gray', interpolation='nearest')
axs[1, 2].set_title("2º Erosión - Sin Cáncer")
axs[1, 2].axis('off')

plt.tight_layout()
plt.show()


Gradiente de Beucher

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Definir el kernel
kernel = np.ones((10, 10), np.uint8)  # Kernel de 10x10

# Función para aplicar dilatación, erosión y calcular gradientes
def compute_gradients(image, kernel):
    image_dilated = cv2.dilate(image, kernel, iterations=1)
    image_eroded = cv2.erode(image, kernel, iterations=1)
    gradient_beucher = image_dilated - image_eroded
    gradient_interno = image - image_eroded
    gradient_externo = image_dilated - image
    return image_dilated, image_eroded, gradient_beucher, gradient_interno, gradient_externo

# Aplicar los cálculos a ambas imágenes
results_cancer = compute_gradients(imagen_mamografia_cancer, kernel)
results_no_cancer = compute_gradients(imagen_mamografia_no_cancer, kernel)

# Desempaquetar resultados
(image_dilated_cancer, image_eroded_cancer, gradient_beucher_cancer,
 gradient_interno_cancer, gradient_externo_cancer) = results_cancer

(image_dilated_no_cancer, image_eroded_no_cancer, gradient_beucher_no_cancer,
 gradient_interno_no_cancer, gradient_externo_no_cancer) = results_no_cancer

# Visualizar los resultados
fig, axs = plt.subplots(3, 4, figsize=(15, 10))

# Títulos comunes
titles = ["Imagen Original", "Dilatación", "Erosión", "Gradiente de Beucher"]

# Imágenes sin cáncer (Primera fila)
images_no_cancer = [imagen_mamografia_no_cancer, image_dilated_no_cancer, image_eroded_no_cancer, gradient_beucher_no_cancer]
for i, (title, img) in enumerate(zip(titles, images_no_cancer)):
    axs[0, i].imshow(img, cmap='gray')
    axs[0, i].set_title(f"{title} - Sin Cáncer")
    axs[0, i].axis('off')

# Imágenes con cáncer (Segunda fila)
images_cancer = [imagen_mamografia_cancer, image_dilated_cancer, image_eroded_cancer, gradient_beucher_cancer]
for i, (title, img) in enumerate(zip(titles, images_cancer)):
    axs[1, i].imshow(img, cmap='gray')
    axs[1, i].set_title(f"{title} - Cáncer")
    axs[1, i].axis('off')

# Gradientes adicionales (interno y externo) para la tercera fila
axs[2, 0].imshow(gradient_interno_cancer, cmap='gray')
axs[2, 0].set_title("Gradiente Interno -  Cáncer")
axs[2, 0].axis('off')

axs[2, 1].imshow(gradient_externo_cancer, cmap='gray')
axs[2, 1].set_title("Gradiente Externo - Cáncer")
axs[2, 1].axis('off')

axs[2, 2].imshow(gradient_interno_no_cancer, cmap='gray')
axs[2, 2].set_title("Gradiente Interno - Sin Cáncer")
axs[2, 2].axis('off')

axs[2, 3].imshow(gradient_externo_no_cancer, cmap='gray')
axs[2, 3].set_title("Gradiente Externo - Sin Cáncer")
axs[2, 3].axis('off')

plt.tight_layout()
plt.show()


Binarización de gradiente de beucher

In [ ]:
# Definir el kernel
kernel = np.ones((10, 10), np.uint8)  # Kernel de 10x10

# Función para calcular gradiente de Beucher y su binarización
def process_beucher(image, kernel, threshold=66):
    # Calcular dilatación y erosión
    image_dilated = cv2.dilate(image, kernel, iterations=1)
    image_eroded = cv2.erode(image, kernel, iterations=1)

    # Calcular gradiente de Beucher
    gradient_beucher = image_dilated - image_eroded

    # Binarización del gradiente
    # Convert gradient_beucher to grayscale before thresholding
    if len(gradient_beucher.shape) == 3:
        gradient_beucher_gray = cv2.cvtColor(gradient_beucher, cv2.COLOR_BGR2GRAY)
    else:
        gradient_beucher_gray = gradient_beucher
    _, binary_gradient = cv2.threshold(gradient_beucher_gray, threshold, 255, cv2.THRESH_BINARY)

    return gradient_beucher, binary_gradient

# Función para calcular áreas de los contornos
def calcular_area_contornos(binary_image):
    # Encontrar los contornos
    contornos, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Calcular el área de cada contorno
    areas = [cv2.contourArea(contorno) for contorno in contornos]

    return areas, contornos

# Aplicar el proceso a ambas imágenes
gradient_beucher_cancer, binary_cancer = process_beucher(imagen_mamografia_cancer, kernel)
gradient_beucher_no_cancer, binary_no_cancer = process_beucher(imagen_mamografia_no_cancer, kernel)

# Calcular áreas de los contornos
areas_cancer, contornos_cancer = calcular_area_contornos(binary_cancer)
areas_no_cancer, contornos_no_cancer = calcular_area_contornos(binary_no_cancer)

# Visualizar los resultados
fig, axs = plt.subplots(2, 3, figsize=(12, 10))

# Resultados para imagen con cáncer
axs[0, 0].imshow(imagen_mamografia_cancer, cmap='gray')
axs[0, 0].set_title("(a) Original - Cáncer")
axs[0, 0].axis('off')

axs[0, 1].imshow(gradient_beucher_cancer, cmap='gray')
axs[0, 1].set_title("(b) Gradiente Beucher - Cáncer")
axs[0, 1].axis('off')

axs[0, 2].imshow(binary_cancer, cmap='gray')
axs[0, 2].set_title("(c) Binarización - Cáncer")
axs[0, 2].axis('off')

# Resultados para imagen sin cáncer
axs[1, 0].imshow(imagen_mamografia_no_cancer, cmap='gray')
axs[1, 0].set_title("(a) Original - Sin Cáncer")
axs[1, 0].axis('off')

axs[1, 1].imshow(gradient_beucher_no_cancer, cmap='gray')
axs[1, 1].set_title("(b) Gradiente Beucher - Sin Cáncer")
axs[1, 1].axis('off')

axs[1, 2].imshow(binary_no_cancer, cmap='gray')
axs[1, 2].set_title("(c) Binarización - Sin Cáncer")
axs[1, 2].axis('off')

plt.tight_layout()
plt.show()

# Imprimir áreas de los contornos
print("Áreas de contornos - Imagen con cáncer:", areas_cancer)
print("Áreas de contornos - Imagen sin cáncer:", areas_no_cancer)


Low Pass Filter + Canny

In [ ]:
edge_color = [255, 0, 0]
low_threshold = 150
high_threshold = 200
kernel_size = 3

imagenes_mamografia = []
imagenes_mamografia.append(imagen_mamografia_cancer)
imagenes_mamografia.append(imagen_mamografia_no_cancer)

titulos = ["Mamografía con Cáncer", "Mamografía sin Cáncer"]

for idx, image in enumerate(imagenes_mamografia):
    image = crop_image(image)
    if len(image.shape) == 3:
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        grayscale_image = image

    original_image = grayscale_image
    lpf_image = filtro_paso_bajo(original_image)

    ssim_value = calcular_ssim(original_image, lpf_image)
    psnr_value = calcular_psnr(original_image, lpf_image)
    print(f"{titulos[idx]}: SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

    edges_original = bordes_canny(original_image)
    edges_lpf = bordes_canny(lpf_image)

    overlay_original = superponer_bordes(original_image, edges_original)
    overlay_lpf = superponer_bordes(lpf_image, edges_lpf)

    plt.figure(figsize=(15, 5))
    plt.suptitle(titulos[idx], fontsize=16, y=1.05)
    plt.subplot(1, 4, 1)
    plt.title(f"Imagen Original")
    plt.imshow(original_image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.title(f"Canny sobre Imagen Original")
    plt.imshow(overlay_original)
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.title(f"Imagen Low Pass Filter")
    plt.imshow(lpf_image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 4, 4)
    plt.title(f"Bordes sobre Imagen Low Pass Filter")
    plt.imshow(overlay_lpf)
    plt.axis('off')

    plt.show()


Filter Box + Canny

In [ ]:
for idx, image in enumerate(imagenes_mamografia):
    image = crop_image(image)
    if len(image.shape) == 3:
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        grayscale_image = image

    original_image = grayscale_image
    filtered_image_box = filtro_box(original_image)

    ssim_value = calcular_ssim(original_image, filtered_image_box)
    psnr_value = calcular_psnr(original_image, filtered_image_box)
    print(f"{titulos[idx]}: SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

    edges_original = bordes_canny(original_image)
    edges_lpf = bordes_canny(filtered_image_box)

    overlay_original = superponer_bordes(original_image, edges_original)
    overlay_box = superponer_bordes(filtered_image_box, edges_lpf)

    plt.figure(figsize=(15, 5))
    plt.suptitle(titulos[idx], fontsize=16, y=1.05)
    plt.subplot(1, 4, 1)
    plt.title(f"Imagen Original")
    plt.imshow(original_image, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.title(f"Canny sobre Imagen Original")
    plt.imshow(overlay_original)
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.title(f"Imagen Box Filter")
    plt.imshow(filtered_image_box, cmap='gray')
    plt.axis('off')

    plt.subplot(1, 4, 4)
    plt.title(f"Bordes sobre Imagen Box Filter")
    plt.imshow(overlay_box)
    plt.axis('off')

    plt.show()


In [ ]:
for idx, image in enumerate(imagenes_mamografia):
  image = crop_image(image)
  original_image = image
  kernel_size = (25, 25)
  closed_image = close(original_image, kernel_size)

  edges_original = cv2.Canny(original_image, low_threshold, high_threshold)
  edges_lpg = cv2.Canny(closed_image, low_threshold, high_threshold)

  if len(original_image.shape) == 2:
      overlay_original = cv2.cvtColor(original_image, cv2.COLOR_GRAY2BGR)
      overlay_lpf = cv2.cvtColor(closed_image, cv2.COLOR_GRAY2BGR)
  else:
      overlay_original = original_image.copy()
      overlay_lpf = closed_image.copy()

  overlay_original[edges_original > 0] = edge_color
  overlay_lpf[edges_lpg > 0] = edge_color

  # Mostrar resultados
  plt.figure(figsize=(15, 5))

  # Subgráfico de la imagen original
  plt.subplot(1, 4, 1)
  plt.title(f"Imagen Original {idx+1}")
  plt.imshow(original_image, cmap='gray')
  plt.axis('off')

  # Subgráfico de bordes sobre la imagen original
  plt.subplot(1, 4, 2)
  plt.title(f"Canny sobre Imagen Original {idx+1}")
  plt.imshow(overlay_original)
  plt.axis('off')

  # Subgráfico de la imagen gradient_beucher
  plt.subplot(1, 4, 3)
  plt.title(f"Imagen gradient_beucher {idx+1}")
  plt.imshow(closed_image, cmap='gray')
  plt.axis('off')

  # Subgráfico de bordes sobre gradient_beucher
  plt.subplot(1, 4, 4)
  plt.title(f"Bordes sobre Imagen gradient_beucher {idx+1}")
  plt.imshow(overlay_lpf)
  plt.axis('off')

  plt.show()

| Filtro                        | SSIM (Cáncer) | PSNR (Cáncer) | SSIM (Sin Cáncer) | PSNR (Sin Cáncer) |
|-------------------------------|---------------|---------------|--------------------|-------------------|
| Sobel X                       | 0.7394        | 13.0344 dB    | 0.3891             | 11.0796 dB       |
| Sobel Y                       | 0.7392        | 13.1620 dB    | 0.3845             | 11.2557 dB       |
| Diagonal 135º                 | 0.8674        | 11.0256 dB    | 0.4934             | 5.1538 dB        |
| Diagonal 45º                  | 0.8672        | 11.0187 dB    | 0.4921             | 5.1447 dB        |
| Laplaciano                    | 0.7228        | 12.9777 dB    | 0.2637             | 10.8934 dB       |
| Prewitt X                     | 0.7444        | 12.9420 dB    | 0.4329             | 11.7742 dB       |
| Prewitt Y                     | 0.7462        | 13.1085 dB    | 0.4310             | 12.0542 dB       |
| Magnitud                      | 0.7475        | 12.7321 dB    | 0.4448             | 11.3883 dB       |
| Low Pass Filter               | 0.9729        | 41.1438 dB    | 0.9066             | 33.0060 dB       |
| Box Filter                    | 0.9707        | 41.1236 dB    | 0.9218             | 33.3629 dB   |


Detección de las imágenes con y sin cáncer, mediante un modelo. https://universe.roboflow.com/frani1999-do9am/cbis-ddsm-oekpf

In [ ]:
#pip install inference-sdk

In [ ]:
from inference_sdk import InferenceHTTPClient, InferenceConfiguration
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image

CLIENT = InferenceHTTPClient(
    api_url="https://detect.roboflow.com",
    api_key="wxFlllAZK77w7KCckQ69"
)
custom_configuration = InferenceConfiguration(confidence_threshold=0.3)

CLIENT.configure(custom_configuration)

for imagen in imagenes_mamografia:
  imagen = crop_image(imagen)
  fig, ax = plt.subplots(1)
  ax.imshow(imagen, cmap='gray')
  try:
    result =  CLIENT.infer(imagen, model_id="cbis-ddsm-oekpf/1")
    predictions = result.get("predictions", [])
    if not predictions:
        print(f"No se encontraron predicciones en la siguiente imagen")
    else:
      for prediction in predictions:
        label = prediction["class"]
        confidence = prediction["confidence"]

        x_center, y_center, width, height = prediction["x"], prediction["y"], prediction["width"], prediction["height"]
        x1 = x_center - width / 2
        y1 = y_center - height / 2

        rect = patches.Rectangle(
            (x1, y1), width, height, linewidth=2, edgecolor='lime', facecolor='none'
        )
        ax.add_patch(rect)
        label = f"{prediction['class']} ({prediction['confidence']:.2f})"
        plt.text(x1, y1 - 10, label, color='lime', fontsize=12, backgroundcolor='black')
    plt.axis("off")
    plt.show()

  except Exception as e:
    print(f"Error procesando {imagen}: {e}")

### IMAGENES SATELITALES

Explicación del procesamiento de las imagenes (objetivo --> detección de zonas verdes):



---

PARTE 1: TÉCNICAS APLICADAS A LAS IMÁGENES.


*  Se convierte la imagen a escala RGB.
*  Se extrae el canal verde de la imagen, esto va a resaltar las intensidades asociadas al color verde, lo es útil en análisis donde este canal tenga mayor relevancia, en este caso queremos detectar zonas verdes.
*  Se aplica un filtro gaussiano con un kernel 5x5 para suavizar la imagen, de esta manera reducimos el ruido y evitamos la detección de detalles innecesarios en etapas posteriores.
*  Se aplica una umbralización mediante el método de Otsu, que calcula el umbral óptimo para separar pixeles claros y oscuros en una imagen.
*  Se aplican dos operaciones morfológicas seguidas, primero un cierre (dialatación y luego erosión) y después una apertura (erosión seguida de dilatación) con un kernel de 3x3 con el objetivo de eliminar el ruido de la imagen y depués mantener sus bordes originales.


---

PARTE 2: EVALUACIÓN Y VISUALIZACIÓN.

* Se calcula el porcentaje de pixeles negros de la imagen (utilizados para cuantificar las zonas verdes).
* Se crea una máscara de los pixeles negros de la imagen (vegetación) y se superpone en la imagen original para visualizar la detección de las zonas verdes.


#### IMAGEN 1

In [ ]:
img1 = cv2.imread('/content/drive/MyDrive/MASTER IA/VISION ARTIFICIAL/ACTIVIDAD 2/bogota1.jpg')

In [ ]:
from skimage.measure import label, regionprops
from skimage.draw import disk

original = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)

# Ajustar tamaño general de la figura
plt.figure(figsize=(20, 15))  # Tamaño ampliado para acomodar imágenes grandes

# Primera fila - Mostrar tres imágenes
# Imagen original
plt.subplot(2, 3, 1)  # 2 filas, 3 columnas, posición 1
plt.title("Original", fontsize=16)
plt.imshow(original, cmap='gray')
plt.axis('off')

# Canal verde
canal_verde = original[:, :, 1]
plt.subplot(2, 3, 2)  # Posición 2
plt.title("Canal Verde", fontsize=16)
plt.imshow(canal_verde, cmap='gray')
plt.axis('off')

# Filtro Gaussiano
gauss = cv2.GaussianBlur(canal_verde, (5, 5), 0)
plt.subplot(2, 3, 3)  # Posición 3
plt.title("Filtro Gaussiano", fontsize=16)
plt.imshow(gauss, cmap='gray')
plt.axis('off')

ssim_value = calcular_ssim(canal_verde, gauss)
psnr_value = calcular_psnr(canal_verde, gauss)

print(f"SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

# Segunda fila - Mostrar dos imágenes
# Umbralización de Otsu
_, umbral_otsu = cv2.threshold(gauss, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
plt.subplot(2, 3, 4)  # Posición 4
plt.title("Umbralización Otsu", fontsize=16)
plt.imshow(umbral_otsu, cmap='gray')
plt.axis('off')

# Operaciones Open y Close
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
imagen_open = cv2.morphologyEx(umbral_otsu, cv2.MORPH_OPEN, kernel)
imagen_close = cv2.morphologyEx(imagen_open, cv2.MORPH_CLOSE, kernel)
plt.subplot(2, 3, 5)  # Posición 5
plt.title("Open + Close", fontsize=16)
plt.imshow(imagen_close, cmap='gray')
plt.axis('off')

# Mostrar todas las imágenes
plt.tight_layout()
plt.show()


labeled_image1 = label(umbral_otsu)
labeled_image2 = label(imagen_open)
labeled_image3 = label(imagen_close)

# Extraer propiedades
props1 = regionprops(labeled_image1)
props2 = regionprops(labeled_image2)
props3 = regionprops(labeled_image3)

total_area1 = sum(region.area for region in props1)
total_area2 = sum(region.area for region in props2)
total_area3 = sum(region.area for region in props3)

print(f"El area total de píxeles de la imagen tras aplicar el umbral otsu es de {total_area1}")
print(f"El area total de píxeles de la imagen tras aplicar la apertura es de {total_area2}")
print(f"El area total de píxeles de la imagen tras aplicar el cierre es de {total_area3}")


In [ ]:
pixeles_negros = np.sum(imagen_close == 0)
total_pixeles = imagen_close.size
porcentaje_negros = (pixeles_negros / total_pixeles) * 100
print(f"Porcentaje de zona verde: {porcentaje_negros:.2f}%")

# Crear una máscara a partir de la imagen de Open + Close
mascara = imagen_close > 0  # Los píxeles negros son False, los blancos son True

# Aplicar la máscara a la imagen original
resultado_nueva_imagen = np.zeros_like(original)  # Crear una imagen vacía del mismo tamaño que la original
resultado_nueva_imagen[mascara] = original[mascara]  # Conservar los píxeles originales donde la máscara es True

# Mostrar las imágenes lado a lado
plt.figure(figsize=(20, 10))

# Imagen original
plt.subplot(1, 2, 1)  # 1 fila, 2 columnas, posición 1
plt.title("Imagen Original", fontsize=16)
plt.imshow(original)
plt.axis('off');

# Imagen con píxeles negros eliminados
plt.subplot(1, 2, 2)  # 1 fila, 2 columnas, posición 2
plt.title("Resultado Final", fontsize=16)
plt.imshow(resultado_nueva_imagen)
plt.axis('off');

plt.tight_layout()
plt.show()

#### IMAGEN 3

In [ ]:
img3 = cv2.imread('/content/drive/MyDrive/MASTER IA/VISION ARTIFICIAL/ACTIVIDAD 2/bogota3.jpg')

In [ ]:
from skimage.measure import label, regionprops
from skimage.draw import disk

original3 = cv2.cvtColor(img3, cv2.COLOR_BGR2RGB)

# Ajustar tamaño general de la figura
plt.figure(figsize=(20, 15))  # Tamaño ampliado para acomodar imágenes grandes

# Primera fila - Mostrar tres imágenes
# Imagen original
plt.subplot(2, 3, 1)  # 2 filas, 3 columnas, posición 1
plt.title("Original", fontsize=16)
plt.imshow(original3, cmap='gray')
plt.axis('off')

# Canal verde
canal_verde3 = original3[:, :, 1]
plt.subplot(2, 3, 2)  # Posición 2
plt.title("Canal Verde", fontsize=16)
plt.imshow(canal_verde3, cmap='gray')
plt.axis('off')

# Filtro Gaussiano
gauss3 = cv2.GaussianBlur(canal_verde3, (5, 5), 0)
plt.subplot(2, 3, 3)  # Posición 3
plt.title("Filtro Gaussiano", fontsize=16)
plt.imshow(gauss3, cmap='gray')
plt.axis('off')

ssim_value = calcular_ssim(canal_verde3, gauss3)
psnr_value = calcular_psnr(canal_verde3, gauss3)

print(f"SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

# Segunda fila - Mostrar dos imágenes
# Umbralización de Otsu
_3, umbral_otsu3 = cv2.threshold(gauss3, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
plt.subplot(2, 3, 4)  # Posición 4
plt.title("Umbralización Otsu", fontsize=16)
plt.imshow(umbral_otsu3, cmap='gray')
plt.axis('off')

# Operaciones Open y Close
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
imagen_open3 = cv2.morphologyEx(umbral_otsu3, cv2.MORPH_OPEN, kernel)
imagen_close3 = cv2.morphologyEx(imagen_open3, cv2.MORPH_CLOSE, kernel)
plt.subplot(2, 3, 5)  # Posición 5
plt.title("Open + Close", fontsize=16)
plt.imshow(imagen_close3, cmap='gray')
plt.axis('off')

# Mostrar todas las imágenes
plt.tight_layout()
plt.show()

labeled_image1 = label(umbral_otsu3)
labeled_image2 = label(imagen_open3)
labeled_image3 = label(imagen_close3)

# Extraer propiedades
props1 = regionprops(labeled_image1)
props2 = regionprops(labeled_image2)
props3 = regionprops(labeled_image3)

total_area1 = sum(region.area for region in props1)
total_area2 = sum(region.area for region in props2)
total_area3 = sum(region.area for region in props3)

print(f"El area total de píxeles de la imagen tras aplicar el umbral otsu es de {total_area1}")
print(f"El area total de píxeles de la imagen tras aplicar la apertura es de {total_area2}")
print(f"El area total de píxeles de la imagen tras aplicar el cierre es de {total_area3}")


In [ ]:
pixeles_negros = np.sum(imagen_close3 == 0)
total_pixeles = imagen_close3.size
porcentaje_negros = (pixeles_negros / total_pixeles) * 100
print(f"Porcentaje de zona verde: {porcentaje_negros:.2f}%")

# Crear una máscara a partir de la imagen de Open + Close
mascara3 = imagen_close3 > 0  # Los píxeles negros son False, los blancos son True

# Aplicar la máscara a la imagen original
resultado_nueva_imagen3 = np.zeros_like(original3)  # Crear una imagen vacía del mismo tamaño que la original
resultado_nueva_imagen3[mascara3] = original3[mascara3]  # Conservar los píxeles originales donde la máscara es True

# Mostrar las imágenes lado a lado
plt.figure(figsize=(20, 10))

# Imagen original
plt.subplot(1, 2, 1)  # 1 fila, 2 columnas, posición 1
plt.title("Imagen Original", fontsize=16)
plt.imshow(original3)
plt.axis('off');

# Imagen con píxeles negros eliminados
plt.subplot(1, 2, 2)  # 1 fila, 2 columnas, posición 2
plt.title("Resultado Final", fontsize=16)
plt.imshow(resultado_nueva_imagen3)
plt.axis('off');

plt.tight_layout()
plt.show()

| Imagen   | Filtro        | SSIM   | PSNR       |
|----------|---------------|--------|------------|
| Imagen 1 | GaussianBlur  | 0.9408 | 31.5440 dB |
| Imagen 2 | GaussianBlur  | 0.9446 | 30.9215 dB |


In [ ]:
def draw_instance_segmentation(image_path, predictions):
    img_rgb = cv2.cvtColor(image_path, cv2.COLOR_BGR2RGB)

    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img_rgb)

    desired_class = ['forest', 'grass', 'field', 'tree', 'farmland']
    for pred in predictions:
        class_name = pred['class']
        #print(f"Class: {class_name}")
        if class_name.strip().lower() in desired_class:

          confidence = pred['confidence']
          points = pred['points']

          polygon = np.array([[point['x'], point['y']] for point in points], dtype=np.int32)
          poly_patch = patches.Polygon(
              polygon, closed=True, edgecolor='lime', facecolor='lime', alpha=0.4
          )
          ax.add_patch(poly_patch)

          '''x_center, y_center, width, height = pred['x'], pred['y'], pred['width'], pred['height']
          x1 = x_center - width / 2
          y1 = y_center - height / 2
          rect = patches.Rectangle(
              (x1, y1), width, height, linewidth=2, edgecolor='yellow', facecolor='none'
          )
          ax.add_patch(rect)

          label = f"{class_name} ({confidence:.2f})"
          plt.text(x1, y1 - 10, label, color='yellow', fontsize=12, backgroundcolor='black')'''

    plt.axis("off")
    plt.show()

imagenes_bogota = []
imagenes_bogota.append(original)
imagenes_bogota.append(original3)

for i in range(len(imagenes_bogota)):
  result = CLIENT.infer(imagenes_bogota[i], model_id="defo_forest-lxfhk/1")
  draw_instance_segmentation(imagenes_bogota[i], result['predictions'])

### IMAGENES INDUSTRIALES

In [ ]:
# dataset: https://www.kaggle.com/datasets/ravirajsinh45/real-life-industrial-dataset-of-casting-product?resource=download
nok = cv2.imread('/content/drive/MyDrive/MASTER IA/VISION ARTIFICIAL ACTIVIDAD 2/imágenes/cast_def_0_5707.jpeg')
ok = cv2.imread('/content/drive/MyDrive/MASTER IA/VISION ARTIFICIAL ACTIVIDAD 2/imágenes/cast_ok_0_1568.jpeg')

plt.figure(figsize=(14, 5))

# Mostrar resultados
plt.subplot(1, 2, 1)
plt.imshow(ok)
plt.title("Imagen ok")
plt.axis("off")

plt.subplot(1, 2, 2)

plt.imshow(nok)
plt.title("Imagen defectuosa")
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
def procesar_imagen(imagen, titulo_fila):
    # Mostrar la imagen original
    plt.subplot(2, 4, titulo_fila * 4 - 3)
    plt.title("Original")
    plt.imshow(imagen, cmap='gray')
    plt.axis('off')

    # Convertir a escala de grises
    gris = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)

    # Aplicar suavizado Gaussiano
    gauss = cv2.GaussianBlur(gris, (5, 5), 0)

    ssim_value = calcular_ssim(gris, gauss)
    psnr_value = calcular_psnr(gris, gauss)

    print(f"SSIM = {ssim_value:.4f}, PSNR = {psnr_value:.4f} dB")

    # Mostrar la imagen suavizada
    plt.subplot(2, 4, titulo_fila * 4 - 2)
    plt.title("Gaussiano")
    plt.imshow(gauss, cmap='gray')
    plt.axis('off')

    # Detectar bordes con Canny
    canny = cv2.Canny(gauss, 50, 150)

    # Mostrar la imagen con bordes
    plt.subplot(2, 4, titulo_fila * 4 - 1)
    plt.title("Canny")
    plt.imshow(canny, cmap='gray')
    plt.axis('off')

    # Buscar contornos
    contornos, _ = cv2.findContours(canny.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Imprimir número de objetos encontrados
    print(f"He encontrado {len(contornos)} objetos en la imagen {titulo_fila}.")

    # Dibujar contornos en la imagen original
    imagen_contornos = imagen.copy()
    cv2.drawContours(imagen_contornos, contornos, -1, (0, 0, 255), 2)

    # Mostrar la imagen con contornos
    plt.subplot(2, 4, titulo_fila * 4)
    plt.title("Contornos")
    plt.imshow(imagen_contornos, cmap='gray')
    plt.axis('off')

# Crear la figura
plt.figure(figsize=(15, 10))

# Procesar ambas imágenes
procesar_imagen(nok, titulo_fila=1)
procesar_imagen(ok, titulo_fila=2)

# Mostrar los resultados
plt.tight_layout()
plt.show()


| Imagen               | SSIM   | PSNR       |
|----------------------|--------|------------|
| Imagen con fallos     | 0.9722 | 38.2577 dB |
| Imagen sin fallos     | 0.9581 | 35.3721 dB |


Explicación del procesamiento de la imagen inferior (objetivo --> detección de defectos en la pieza mala):


---

PARTE 1: TÉCNICAS APLICADAS A LAS IMAGENES.

*  Se convierte la imagen a escala de grises.
*  Se aplica un filtro gaussiano con un kernel 5x5 para suavizar la imagen, de esta manera reducimos el ruido y evitamos la detección de detalles innecesarios en etapas posteriores.
*  Se aplica una umbralización adaptativa basada en una ventana con pesos definidos por una distribución Gaussiana, técnica para binarizar imágenes con iluminación variante en las áreas de la imagen.
*  Se aplican dos operaciones morfológicas seguidas, primero un cierre (dialatación y luego erosión) y después una apertura (erosión seguida de dilatación) con un kernel de 3x3 con el objetivo de eliminar el ruido de la imagen y depués mantener sus bordes originales.
* Por último se invierte la imagen con un operador not, se buscan los contornos y se filtran por tamaño en píxeles que ocupen.

PARTE 2: EVALUALUACIÓN Y VISUALIZACIÓN
* Se concluye que la imagen con la pieza defectuosa tiene mayor numero de contornos encontrados que la pieza buena filtrando por tamaño de pixeles.
* Para cuantificar los resultados se resta 3 al numero total de contornos encontrados para obviar las dos zonas negras interiores de la pieza y la sombra exterior propia de la pieza.
* Se dibujan los contornos encontrados en la imagen original.




In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage.measure import label, regionprops
from skimage.draw import disk

def procesar_imagen(imagen, umbral_area=250):
    # Crear una figura para mostrar las imágenes
    plt.figure(figsize=(15, 6))

    # Mostrar la imagen original
    plt.subplot(1, 6, 1)
    plt.title("Original")
    plt.imshow(imagen, cmap='gray')
    plt.axis('off')

    # Convertir la imagen a escala de grises
    gris = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)

    # Aplicar suavizado Gaussiano
    gauss = cv2.GaussianBlur(gris, (5, 5), 0)

    # Mostrar la imagen suavizada
    plt.subplot(1, 6, 2)
    plt.title("Gaussiano")
    plt.imshow(gauss, cmap='gray')
    plt.axis('off')

    # Aplicar umbral adaptativo
    umbral_adaptativo = cv2.adaptiveThreshold(gauss, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

    # Mostrar la imagen umbralizada
    plt.subplot(1, 6, 3)
    plt.title("Umbral Adaptativo")
    plt.imshow(umbral_adaptativo, cmap='gray')
    plt.axis('off')

    labeled_image1 = label(umbral_adaptativo)

    # Extraer propiedades
    props1 = regionprops(labeled_image1)

    total_area1 = sum(region.area for region in props1)

    print(f"El total de píxeles de la imagen tras aplicar el umbral adaptativo es de {total_area1}")

    # Kernel de 3x3 para las operaciones morfológicas
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    # Primer cierre (Close) - dilatación seguida de erosión
    cierre1 = cv2.morphologyEx(umbral_adaptativo, cv2.MORPH_CLOSE, kernel)

    labeled_image2 = label(cierre1)

    # Extraer propiedades
    props2 = regionprops(labeled_image2)

    total_area2 = sum(region.area for region in props2)

    print(f"El total de píxeles de la imagen tras aplicar el cierre es de {total_area2}")


    # Mostrar la imagen después del primer cierre
    plt.subplot(1, 6, 4)
    plt.title("Cierre (Close)")
    plt.imshow(cierre1, cmap='gray')
    plt.axis('off')

    # Primera apertura (Open) - erosión seguida de dilatación
    apertura1 = cv2.morphologyEx(cierre1, cv2.MORPH_OPEN, kernel)

    labeled_image3 = label(apertura1)

    # Extraer propiedades
    props3 = regionprops(labeled_image3)

    total_area3 = sum(region.area for region in props3)

    print(f"El total de píxeles de la imagen tras aplicar la apertura es de {total_area3}")

    # Mostrar la imagen después de la primera apertura
    plt.subplot(1, 6, 5)
    plt.title("Apertura (Open)")
    plt.imshow(apertura1, cmap='gray')
    plt.axis('off')

    # Invertir directamente la imagen binarizada (manchas negras -> blancas, y viceversa)
    umbralizada_negra = cv2.bitwise_not(apertura1)

    # Encontrar los contornos de las manchas negras (ahora invertidas)
    contornos_negras, _ = cv2.findContours(umbralizada_negra.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar los contornos con un área menor al umbral
    contornos_filtrados = [contorno for contorno in contornos_negras if cv2.contourArea(contorno) >= umbral_area]

    # Contar el número de manchas detectadas
    num_manchas = len(contornos_filtrados)

    # Mostrar el número de manchas detectadas
    print(f"Se han detectado {num_manchas - 3} defectos en la pieza.")

    for contorno in contornos_negras:
       if cv2.contourArea(contorno) >= umbral_area:
        print(cv2.contourArea(contorno))

    # Crear una imagen para dibujar los contornos de las manchas negras
    imagen_contornos_negras = imagen.copy()

    # Dibujar los contornos filtrados de las manchas negras sobre la imagen original
    cv2.drawContours(imagen_contornos_negras, contornos_filtrados, -1, (0, 255, 0), 2)

    # Mostrar la imagen con las manchas negras y sus contornos
    plt.subplot(1, 6, 6)
    plt.title(f"Manchas Negras (> {umbral_area} píxeles)")
    plt.imshow(imagen_contornos_negras)
    plt.axis('off')

    # Mostrar todas las imágenes
    plt.tight_layout()
    plt.show()

# Crear la figura
plt.figure(figsize=(15, 10))

# Procesar ambas imágenes
procesar_imagen(nok)
procesar_imagen(ok)


